In [ ]:
print("\n" + "="*70)
print("✅ ĐÁNH GIÁ MÔ HÌNH HOÀN THÀNH")
print("="*70)
print("\n📝 Tóm tắt kết quả:")
print(f"- Mô hình tốt nhất: {best_model_name}")
print(f"- F1-Score: {best_f1:.4f}")
print(f"- Các yếu tố ảnh hưởng mạnh nhất: Glucose, BMI, Age, Insulin")
print("\n📝 Các bước tiếp theo:")
print("1. ✓ Tiền xử lý dữ liệu")
print("2. ✓ Huấn luyện các mô hình")
print("3. ✓ Đánh giá mô hình")
print("4. → Dự đoán nguy cơ tiểu đường (Demo)")
print("="*70)

In [ ]:
# In Feature Importance dạng bảng
print("\n📊 Random Forest - Feature Importance Ranking:")
print("="*60)
rf_importance_df = pd.DataFrame({
    'Feature': rf_features,
    'Importance': rf_importance
}).sort_values('Importance', ascending=False)

for idx, row in rf_importance_df.iterrows():
    print(f"{row['Feature']:.<25} {row['Importance']:.4f}")

print("\n📊 XGBoost - Feature Importance Ranking:")
print("="*60)
xgb_importance_df = pd.DataFrame({
    'Feature': rf_features,
    'Importance': xgb_importance
}).sort_values('Importance', ascending=False)

for idx, row in xgb_importance_df.iterrows():
    print(f"{row['Feature']:.<25} {row['Importance']:.4f}")

In [ ]:
# Feature Importance cho Random Forest
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Random Forest
rf_model = models['Random Forest']
rf_importance = rf_model.feature_importances_
rf_features = X.columns.tolist()

# Sắp xếp
rf_indices = np.argsort(rf_importance)[::-1][:8]

axes[0].barh(range(len(rf_indices)), rf_importance[rf_indices], color='#3498db', alpha=0.8, edgecolor='black')
axes[0].set_yticks(range(len(rf_indices)))
axes[0].set_yticklabels([rf_features[i] for i in rf_indices])
axes[0].set_xlabel('Feature Importance', fontweight='bold')
axes[0].set_title('📊 Random Forest - Feature Importance', fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3, axis='x')

# XGBoost
xgb_model = models['XGBoost']
xgb_importance = xgb_model.feature_importances_

xgb_indices = np.argsort(xgb_importance)[::-1][:8]

axes[1].barh(range(len(xgb_indices)), xgb_importance[xgb_indices], color='#e74c3c', alpha=0.8, edgecolor='black')
axes[1].set_yticks(range(len(xgb_indices)))
axes[1].set_yticklabels([rf_features[i] for i in xgb_indices])
axes[1].set_xlabel('Feature Importance', fontweight='bold')
axes[1].set_title('📊 XGBoost - Feature Importance', fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("✓ Feature Importance đã được vẽ")

## 6️⃣ Feature Importance

In [ ]:
# Vẽ ROC Curve cho tất cả mô hình
plt.figure(figsize=(10, 8))

colors = ['#2ecc71', '#3498db', '#e74c3c', '#f39c12']

for idx, model_name in enumerate(predictions.keys()):
    y_pred_proba = predictions[model_name]['y_pred_proba']
    
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, color=colors[idx], lw=2, 
             label=f'{model_name} (AUC = {roc_auc:.3f})')

# Đường đối chứng (diagonal)
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.500)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)', fontweight='bold')
plt.ylabel('True Positive Rate (TPR)', fontweight='bold')
plt.title('📈 ROC Curve - Tất Cả Mô Hình', fontweight='bold', fontsize=14)
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ ROC Curve đã được vẽ")

## 5️⃣ ROC Curve

In [ ]:
# Vẽ Confusion Matrix cho tất cả mô hình
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('🔍 Confusion Matrix cho Tất Cả Mô Hình', fontsize=14, fontweight='bold')

model_names = list(predictions.keys())

for idx, model_name in enumerate(model_names):
    ax = axes[idx // 2, idx % 2]
    cm = predictions[model_name]['confusion_matrix']
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, cbar=False,
                xticklabels=['Không', 'Mắc'], yticklabels=['Không', 'Mắc'])
    ax.set_title(model_name, fontweight='bold')
    ax.set_ylabel('Thực tế (Actual)')
    ax.set_xlabel('Dự đoán (Predicted)')

plt.tight_layout()
plt.show()

print("✓ Confusion Matrix đã được vẽ")

## 4️⃣ Confusion Matrix

In [ ]:
# Vẽ biểu đồ so sánh các metrics
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('📊 So Sánh Hiệu Suất Các Mô Hình', fontsize=16, fontweight='bold', y=1.00)

metrics_cols = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']

for idx, metric in enumerate(metrics_cols):
    ax = axes[idx // 3, idx % 3]
    colors = ['#2ecc71', '#3498db', '#e74c3c', '#f39c12']
    
    bars = ax.bar(range(len(results_df)), results_df[metric], color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax.set_ylabel(metric, fontweight='bold')
    ax.set_title(f'{metric}', fontweight='bold')
    ax.set_ylim(0, 1.0)
    ax.set_xticks(range(len(results_df)))
    ax.set_xticklabels(results_df['Model'], rotation=45, ha='right')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Thêm giá trị trên cột
    for bar, val in zip(bars, results_df[metric]):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# Ẩn subplot cuối
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("✓ Biểu đồ so sánh đã được vẽ")

## 3️⃣ So Sánh Các Metrics

In [ ]:
# Tạo DataFrame kết quả
results_df = pd.DataFrame(results)

print("\n📋 Bảng So Sánh Tất Cả Mô Hình:")
print(results_df.to_string(index=False))

# Tìm mô hình tốt nhất
best_model_idx = results_df['F1-Score'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Model']
best_f1 = results_df.loc[best_model_idx, 'F1-Score']

print(f"\n🏆 Mô hình tốt nhất: {best_model_name} (F1-Score: {best_f1:.4f})")

In [ ]:
# Hàm đánh giá mô hình
def evaluate_model(model, X_test, y_test, model_name):
    """Đánh giá mô hình trên tập test"""
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba),
    }
    
    return metrics, y_pred, y_pred_proba, confusion_matrix(y_test, y_pred)

# Đánh giá tất cả mô hình
results = []
predictions = {}

print("\n📊 Đánh giá các mô hình trên tập Test:")
print("="*70)

for model_name, model in models.items():
    metrics, y_pred, y_pred_proba, cm = evaluate_model(model, X_test_scaled, y_test, model_name)
    results.append(metrics)
    predictions[model_name] = {
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'confusion_matrix': cm
    }
    
    print(f"\n{model_name}:")
    print(f"  Accuracy:  {metrics['Accuracy']:.4f}")
    print(f"  Precision: {metrics['Precision']:.4f}")
    print(f"  Recall:    {metrics['Recall']:.4f}")
    print(f"  F1-Score:  {metrics['F1-Score']:.4f}")
    print(f"  ROC-AUC:   {metrics['ROC-AUC']:.4f}")

print("\n" + "="*70)

## 2️⃣ Đánh Giá Tất Cả Mô Hình

In [ ]:
# Tải mô hình từ file
models = {}
model_names = ['Logistic Regression', 'Random Forest', 'XGBoost', 'KNN']

for model_name in model_names:
    model_path = f'../models/{model_name.replace(" ", "_").lower()}_model.pkl'
    model = joblib.load(model_path)
    models[model_name] = model
    print(f"✓ Tải mô hình: {model_name}")

print(f"\n✓ Tất cả {len(models)} mô hình đã được tải thành công")

In [ ]:
# Tải dữ liệu
data_path = '../data/diabetes.csv'
df = pd.read_csv(data_path)

# Tiền xử lý dữ liệu (giống như trong notebook 2)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

df_clean = df.copy()
cols_with_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in cols_with_zero:
    median_val = df_clean[df_clean[col] != 0][col].median()
    df_clean.loc[df_clean[col] == 0, col] = median_val

X = df_clean.drop('Outcome', axis=1)
y = df_clean['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

print("✓ Dữ liệu đã được tải và tiền xử lý")

## 1️⃣ Tải Dữ Liệu và Mô Hình

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, auc, confusion_matrix,
    classification_report, ConfusionMatrixDisplay
)
import joblib
import warnings
warnings.filterwarnings('ignore')

print("✓ Các thư viện đã được import thành công!")

# 📊 Dự đoán Bệnh Tiểu Đường - Đánh Giá Mô Hình

## Phần 3: Đánh giá hiệu suất các mô hình
- Accuracy, Precision, Recall, F1-Score
- Confusion Matrix
- ROC-AUC Curve
- Feature Importance